# Notebook for importing from the Unihan Database

In [39]:
# Assumes the Unihan database has been downloaded from http://www.unicode.org/Public/UCD/latest/ucd/Unihan.zip
# and placed in the directory ../Unihan
import sys

import pandas as pd
import numpy as np

# Load Unihan Readings table
colnames = ['codepoint', 'fieldname', 'value']
types = {'codepoint': np.string_, 'fieldname': np.string_, 'value': np.string_}
readings = pd.read_table('../Unihan/Unihan_Readings.txt', names=colnames, dtype=types)
readings.set_index("codepoint")
print readings['codepoint'].count() # Number of rows

# Load Unihan Variants table
colnames = ['codepoint', 'fieldname', 'value']
types = {'codepoint': np.string_, 'fieldname': np.string_, 'value': np.string_}
variants = pd.read_table('../Unihan/Unihan_Variants.txt', names=colnames, dtype=types)
print variants['codepoint'].count() # Number of rows

readings.tail()

187561
12375


,codepoint,fieldname,value
187556,U+2F994,kCantonese,fong1
187557,U+2F9B2,kCantonese,kwai4
187558,U+2F9BC,kCantonese,sip3
187559,U+2F9D4,kCantonese,gun3 gwun3
187560,# EOF,NaN,NaN


In [43]:
chinese = u'伝'
cp = 'U+' + hex(ord(chinese)).replace('0x', '').upper()
chineseDf = readings[readings.codepoint == cp]
chineseDf

,codepoint,fieldname,value
24361,U+4F1D,kDefinition,"summon; propagate, transmit"
24362,U+4F1D,kJapaneseKun,TSUTAERU TSUTAE TSUTAWARU
24363,U+4F1D,kJapaneseOn,DEN TEN
24364,U+4F1D,kKorean,CEN
24365,U+4F1D,kMandarin,yún


In [44]:
variantsDf = variants[variants.codepoint == cp]
variantsDf

,codepoint,fieldname,value
598,U+4F1D,kZVariant,U+50B3


In [45]:
english = "\\N"
englishDF = chineseDf[chineseDf.fieldname == 'kDefinition']['value']
if len(englishDF) > 0:
    english = englishDF.iloc[0]
print english

pinyin = chineseDf[chineseDf.fieldname == 'kMandarin']['value'].iloc[0].decode('utf-8')
print pinyin

# Kinds of variants
notes = ""
c = ""
value = variantsDf[variantsDf.fieldname == 'kSemanticVariant']['value']
if len(value) > 0:
    val = value.iloc[0].split('<')[0]
    val = val.replace("U+", "")
    c = unichr(int(val, 16))
    notes = "Semantic variant: %s " % c
print notes

simplified = chinese
traditional = "\\N"
value = variantsDf[variantsDf.fieldname == 'kSimplifiedVariant']['value']
if len(value) > 0:
    val = value.iloc[0].split('<')[0]
    val = val.replace("U+", "")
    simplified = unichr(int(val, 16))
    traditional = chinese
print simplified

grammar = "\\N"
if english != "\\N":
    grammar = "noun"
print grammar

luid = 57940
notes = "%s (Unihan '%s')" % (notes, chinese)
domain = u"古文\tClassical Chinese"
print u"%d\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t\\N\t\\N\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, traditional, pinyin, english, grammar, domain, notes, luid)

summon; propagate, transmit
yún

伝
noun
57940	伝	\N	yún	summon; propagate, transmit	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	 (Unihan '伝')	57940


In [27]:
# Generate lexical entries for the characters in file unknown.txt
luid = 66853
colnames = ['codepoint', 'char']
types = {'char': np.string_, 'codepoint': np.string_}
unknownDF = pd.read_table('unknown.txt', names = colnames, dtype=types, header = None)
unknownDF

,codepoint,char


In [28]:
unknownReadings = pd.merge(unknownDF, readings, on = "codepoint")
unknownReadings.set_index("codepoint")
unknownReadings

,char,codepoint,fieldname,value


In [29]:
pivoted = unknownReadings.pivot("codepoint", "fieldname", "value")
del pivoted["kCantonese"]
del pivoted["kHanyuPinyin"]
del pivoted["kJapaneseKun"]
del pivoted["kJapaneseOn"]
del pivoted["kVietnamese"]
del pivoted["kXHC1983"]
pivoted = pivoted.rename(columns = {"kDefinition":"english", "kMandarin":"pinyin"})
pivoted

KeyError: 'kCantonese'

In [8]:
pivoted = pivoted.fillna("\\N")
pivoted

fieldname
codepoint


In [9]:
unknownDF = unknownDF.join(pivoted, on = "codepoint", lsuffix = "_")
unknownDF

,codepoint,char


In [10]:
unknownDF['id'] = range(luid, luid + len(unknownDF))
unknownDF['traditional'] = "\\N"
getGrammar = lambda x: "noun" if x.english != "\\N" else "\\N"
unknownDF['grammar'] = unknownDF.apply(getGrammar, axis = 1)
unknownDF['concept_cn'] = "\\N"
unknownDF['concept_en'] = "\\N"
unknownDF['domain_cn'] = "古文"
unknownDF['domain_en'] = "Classical Chinese"
unknownDF['subdomain_cn'] = "\\N"
unknownDF['subdomain_en'] = "\\N"
unknownDF['mp3'] = "\\N"
unknownDF['image'] = "\\N"
writeNotes = lambda c: "(Unihan '" + c + "')"
unknownDF['notes'] = unknownDF['char'].map(writeNotes)
unknownDF['headword'] = unknownDF['id']
unknownDF

/Library/Python/2.7/site-packages/pandas/indexes/range.py:439: RuntimeWarning: tp_compare didn't return -1 or -2 for exception
  return max(0, -(-(self._stop - self._start) // self._step))


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [106]:
cols = ['id', 'char', 'traditional', 'pinyin', 'english', 'grammar', 'concept_cn', 'concept_en', 'domain_cn', 'domain_en', 'subdomain_cn', 'subdomain_en', 'mp3', 'image', 'notes', 'headword']
unknownDF = unknownDF[cols]
unknownDF.to_csv(sys.stdout, index = False, header = False, sep = "\t", cols = cols)

61731	㡏	\N	shù	to cut a strip of cloth; a breadth of material, (same as 繻) fine gauze, frayed edges of silk, silk torn into two pieces, one of which was given as a credential and the other retained, a loose garment or cloak; fine clothes, the left over material after cutting; ragged fabric	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '㡏')	61731
61732	𨪉	\N	tí	\N	\N	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𨪉')	61732
61733	𣚍	\N			noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𣚍')	61733
61734	谫	\N	jiǎn	shallow; stupid	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '谫')	61734
61735	𨓆	\N			noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𨓆')	61735
61736	㞓	\N	qì	the buttocks; the rump; the sacrum, to set sideways, the male organ; (Cant.) vulgar term for the female sex organ	noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '㞓')	61736
61737	𥵸	\N			noun	\N	\N	古文	Classical Chinese	\N	\N	\N	\N	(Unihan '𥵸')	61737
61738	𧾨	\N	xiàn	\N	\N	\N	\N	古文	Classical Chinese	\N	\N